In [55]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [56]:
df = pd.read_csv('tatamotors.csv')
df.head()

,Volume,Next day %,High %,Low %,Close %
0,400353.0,-0.714279,0.357145,-0.714279,-0.714279
1,226686.0,1.785717,1.785717,0.000000,1.071426
2,318529.0,1.052629,1.052629,-0.350880,0.701749
3,1173507.0,3.125001,3.472227,-0.347225,2.430552
4,681310.0,0.000000,1.010099,-1.683503,-0.336703


In [57]:
X = df.drop(columns='Next day %')
y = df['Next day %']

In [58]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=42)

In [59]:
X_train

,Volume,High %,Low %,Close %
710,12741962.0,8.230570,-0.268093,7.292234
5545,12629045.0,2.130109,-0.287853,0.759928
2133,40642105.0,2.505487,-1.916558,1.587140
558,6589761.0,4.894040,-0.289685,3.750565
5947,42250518.0,1.470593,-3.114185,0.951556
...,...,...,...,...
3772,15690758.0,2.860266,-1.113535,2.091702
5191,9476008.0,2.315855,-0.306060,2.019995
5226,4912743.0,0.799700,-1.385533,-1.069370
5390,4253044.0,0.518135,-2.217619,-0.207254


In [60]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
trf1=ColumnTransformer([
    ('scaler',MinMaxScaler(),[0])
],remainder='passthrough')

In [61]:
trf2 = LinearRegression()

In [62]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2)
])

In [63]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  [0])])),
                ('trf2', LinearRegression())])

In [64]:
X_test.values[0]

array([ 9.05114200e+06,  1.91431816e+00, -3.57037485e-02,  1.18696484e+00])

In [65]:
y_test

3921    1.847384
1295   -2.981971
4693    0.943195
2795   -1.025995
408    -0.800000
          ...   
4988    2.964958
5111   -0.412780
530    -3.361347
5469   -1.738618
5660   -2.333932
Name: Next day %, Length: 1414, dtype: float64

In [66]:
y_pred = pipe.predict(X_test)

In [67]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [68]:
print("MAE",mean_absolute_error(y_test,y_pred))
print("MSE",mean_squared_error(y_test,y_pred))
print("R2 score",r2_score(y_test,y_pred))

MAE 0.8284878599391541
MSE 1.9535567092230384
R2 score 0.774441077706658


In [69]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))